# 000 Forecasting Bot

Starting from https://colab.research.google.com/drive/1_Il5h2Ed4zFa6Z3bROVCE68LZcSi4wHX?usp=sharing

## Imports

In [1]:
from IPython.display import Markdown

## Today

In [2]:
import datetime
today = str(datetime.datetime.now())[0:10]
today

'2024-08-02'

## 000_bot

### API Keys

In order to run this notebook as is, you'll need to enter a few API keys (use the key icon on the left to input them):

- `METACULUS_TOKEN`: you can find your Metaculus token under your bot's user settings page: https://www.metaculus.com/accounts/settings/, or on the bot registration page where you created the account: https://www.metaculus.com/aib/
- `OPENAPI_API_KEY`: get one from OpenAIs page: https://platform.openai.com/settings/profile?tab=api-keys
- `PERPLEXITY_API_KEY` - used to search up-to-date information about the question. Get one from https://www.perplexity.ai/settings/api
- `ASKNEWS_CLIENT_ID`, `ASKNEWS_SECRET`

In [3]:
from omegaconf import OmegaConf
token_fn = "tokens.yaml"

In [4]:
config = OmegaConf.load(token_fn)

def pr(tokens):
    print(OmegaConf.to_yaml(config))

### LLM and Metaculus Interaction

This section sets up some simple helper code you can use to get data about forecasting questions and to submit a prediction

In [5]:
import datetime
import json
import os
import requests
import re
from openai import OpenAI
from tqdm import tqdm

In [6]:
AUTH_HEADERS = {"headers": {"Authorization": f"Token {config.METACULUS_TOKEN}"}}
API_BASE_URL = "https://www.metaculus.com/api2"
WARMUP_TOURNAMENT_ID = 3349
SUBMIT_PREDICTION = True

def find_number_before_percent(s):
    # Use a regular expression to find all numbers followed by a '%'
    matches = re.findall(r'(\d+)%', s)
    if matches:
        # Return the last number found before a '%'
        return int(matches[-1])
    else:
        # Return None if no number found
        return None

def post_question_comment(question_id, comment_text):
    """
    Post a comment on the question page as the bot user.
    """

    response = requests.post(
        f"{API_BASE_URL}/comments/",
        json={
            "comment_text": comment_text,
            "submit_type": "N",
            "include_latest_prediction": True,
            "question": question_id,
        },
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    print("Comment posted for ", question_id)

def post_question_prediction(question_id, prediction_percentage):
    """
    Post a prediction value (between 1 and 100) on the question.
    """
    url = f"{API_BASE_URL}/questions/{question_id}/predict/"
    response = requests.post(
        url,
        json={"prediction": float(prediction_percentage) / 100},
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    print("Prediction posted for ", question_id)


def get_question_details(question_id):
    """
    Get all details about a specific question.
    """
    url = f"{API_BASE_URL}/questions/{question_id}/"
    response = requests.get(
        url,
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    return json.loads(response.content)

def list_questions(tournament_id=WARMUP_TOURNAMENT_ID, offset=0, count=1000):
    """
    List (all details) {count} questions from the {tournament_id}
    """
    url_qparams = {
        "limit": count,
        "offset": offset,
        "has_group": "false",
        "order_by": "-activity",
        "forecast_type": "binary",
        "project": tournament_id,
        "status": "open",
        "type": "forecast",
        "include_description": "true",
    }
    url = f"{API_BASE_URL}/questions/"
    response = requests.get(url, **AUTH_HEADERS, params=url_qparams)
    response.raise_for_status()
    data = json.loads(response.content)
    return data

### IFP

In [7]:
class IFP:

    def __init__(self, question_id):
        self.question_id = question_id
        self.question_details = get_question_details(self.question_id)
        self.today = datetime.datetime.now().strftime("%Y-%m-%d")   
        self.title = self.question_details["title"]
        self.resolution_criteria = self.question_details["resolution_criteria"]
        self.background = self.question_details["description"]
        self.fine_print = self.question_details["fine_print"]

    def report(self):
        rpt = f"""
The future event is described by this question: [ {self.title} ]
The resolution criteria are: [ {self.resolution_criteria} ]
The background is: [ {self.background} ]"""
        if self.fine_print:
            rpt += f"""
The fine print is: [ {self.fine_print} ]"""
        return rpt

### LLMs

### In general

In [8]:
class LLM:
    def __init__(self, system_role):
        self.messages = [{"role": "system", "content": system_role}]

    def chat(self, query):
        self.messages.append({"role": "user", "content": query})
        text = self.message()
        self.messages.append({"role": "assistant", "content": text})
        return text

### MetaAI

https://pypi.org/project/meta-ai-api/1.0.6/

In [9]:
 from meta_ai_api import MetaAI as mai

In [10]:
class MetaAI(LLM):

    def __init__(self):
        self.ai = mai()
        super().__init__('MetaAI')
        
    def message(self):
        return self.ai.prompt(message=self.messages[-1]['content'])

### HuggingChat

https://pypi.org/project/hugchat/

In [11]:
from hugchat import hugchat
from hugchat.login import Login

In [12]:
class HuggingChat(LLM):

    def __init__(self):
        # Log in to huggingface and grant authorization to huggingchat
        EMAIL = config.HUGGINGFACE_USERNAME
        PASSWD = config.HUGGINGFACE_PASSWORD
        cookie_path_dir = "./cookies/" # NOTE: trailing slash (/) is required to avoid errors
        sign = Login(EMAIL, PASSWD)
        cookies = sign.login(cookie_dir_path=cookie_path_dir, save_cookies=True)
        self.chatbot = hugchat.ChatBot(cookies=cookies.get_dict()) 
        super().__init__('HuggingChat')
        
    def message(self):
        result = ''.join([x['token'] for x in self.chatbot.chat(self.messages[-1]['content']) if x])
        return result

    def web_search(self, query):
        query_result = self.chatbot.query(query, web_search=True)
        return query_result
        print(query_result)
        for source in query_result.web_search_sources:
            print(source.link)
            print(source.title)
            print(source.hostname)

    def available_models(self):
        return [(i,str(x)) for i,x in enumerate(self.chatbot.get_available_llm_models())]

    def switch_llm(self, i):
        self.chatbot.switch_llm(i)

    def conversation_info(self):
        info = self.chatbot.get_conversation_info()
        # print(info.id, info.title, info.model, info.system_prompt, info.history)
        return info

In [13]:
hc = HuggingChat()

### Metaculus Anthropic

In [41]:
import json

class Claude(LLM):

    def __init__(self):
        super().__init__('Claude')
        
    def message(self):
        content = self.messages[-1]['content']
        url = "https://www.metaculus.com/proxy/anthropic/v1/messages"
        headers =  {"Authorization": f"Token {config.METACULUS_TOKEN}",
                    "anthropic-version": "2023-06-01",
                    "content-type": "application/json"}
        data = {"model": "claude-3-5-sonnet-20240620",
                "max_tokens": 1024,
                "messages": [{"role": "user", "content": content}]}
        response = requests.post(url, headers=headers, data = json.dumps(data))
        # Check the response status code
        if response.status_code != 200:
            print("Request failed with status code", response.status_code)
            print(response.text)
        return response.json()['content'][0]['text']

### AskNews

https://colab.research.google.com/drive/1tc383HraMZOiyfKFF1EXAtlTYbsuv3Q5?usp=sharing

In [48]:
from asknews_sdk import AskNewsSDK

In [60]:
import json

class AskNews(LLM):

    def __init__(self):
        super().__init__('AskNews')
        ASKNEWS_CLIENT_ID = config.ASKNEWS_CLIENT_ID
        ASKNEWS_SECRET = config.ASKNEWS_SECRET
        
        self.ask = AskNewsSDK(
              client_id=config.ASKNEWS_CLIENT_ID,
              client_secret=config.ASKNEWS_SECRET,
              scopes=["news"]
          )
        
    def message(self):
        query = self.messages[-1]['content']
        return self.ask.news.search_news(query).as_string

In [67]:
news = ask.chat('Bruce Bueno de Mesquita')

### Perplexity

In [70]:
class Perplexity(LLM):
    def message(self):
        url = "https://api.perplexity.ai/chat/completions"
        headers = {
            "accept": "application/json",
            "authorization": f"Bearer {config.PERPLEXITY_API_KEY}",
            "content-type": "application/json"  }
        payload = {"model": config.PERPLEXITY_MODEL, "messages": self.messages }
        response = requests.post(url=url, json=payload, headers=headers)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]

### Metaculus OpenAI

### OpenAI

In [84]:
class ChatGPT(LLM):
    def __init__(self, system_role):
        super().__init__(system_role)
        self.client = OpenAI(api_key=config.OPENAI_API_KEY)

    def message(self):
        chat_completion = self.client.chat.completions.create(
            model=config.OPENAI_MODEL,
            messages= self.messages)
        return chat_completion.choices[0].message.content

### Test questions

### Agents

In [85]:
class Agent:
    def __init__(self, system_role, llm):
        self.llm = llm(system_role)

    def chat(self, prompt):
        return self.llm.chat(prompt)

#### Rate Analyzer(TBD: ChatGPT is better)

#### Newser

In [123]:
class Newser(Agent):
    def __init__(self):
        self.ask = AskNews()

    def recherche(self, ifp):
        ifp.news = self.ask.chat(ifp.title)

    def research(self, ifps):
        for ifp in tqdm(ifps):
            self.recherche(ifps[ifp])

#### Researcher

In [106]:
class Researcher(Agent):
    def __init__(self, llm):
        self.system_role = f"""
You are an open source intelligence analyst.
You summarize news related to questions about events.
A question about an event is formatted as |id|question|criteria|background|fineprint|.
You will find on the web and report any reliable information you can gather about the question.
Do not make an assessment of probability.
Do not repeat information provided to you already in the prompt."""
        super().__init__(self.system_role, llm)

    def recherche(self, ifp):
        prompt = f"|{ifp.question_id}|{ifp.title}|{ifp.resolution_criteria}|{ifp.background}|{ifp.fine_print}|"
        self.R = self.chat(prompt)
        ifp.news = self.R
        print(ifp.question_id, ifp.title)
        print(self.R)

    def research(self, ifps):
        for ifp in tqdm(ifps):
            self.recherche(ifps[ifp])

### Question relator

In [107]:
class QuestionRelator(Agent):
    def __init__(self, llm):
        self.system_role = f"""
You are prompted with list of forecasting questions, each with an id and a title.
Label each question with an underlying event.
If the questions are for the same event, use the same name for the underlying event.
Please return as separate lines formatted as |event|id|title|, do not add any other formatting.
"""
        super().__init__(self.system_role, llm)

    def relate(self, ifps):
        prompt = '\n'.join([f"{ifp.question_id}: {ifp.title}" for ifp in ifps.values()])
        KL = self.chat(prompt)
        K1 = [x.split('|') for x in KL.split('\n')]
        K2 = [(int(id),event) for _,event,id,_,_ in K1]
        for id,event in K2:
            ifps[id].event = event
            print(id, event)

#### Superforecaster

In [108]:
class Superforecaster(Agent):
    def __init__(self, llm):
        self.system_role = f"""
You are a superforecaster.  
You assign a probability to questions about events.
Questions are given as separate groups of lines formatted as |FORECAST|event|id|question|news|criteria|background|fineprint|.
Groups are separated by '^^^'.
Questions which are about the same event should be assigned consistent probabilities.
Reply to questions with |ASSESSMENT|id|ZZ|rationale| where ZZ is an integer probability from 1 to 99 and rationale is your reasoning for the forecast.
Separate each question with '^^^'.
Do not add any additional headings or group labels or other formatting.
After your initial forecast you may receive feedback of form |CRITIC|id|feedback|.
Reply to each feedback with |id|ZZ|rationale| where ZZ is an integer probability from 1 to 99 and 
and rationale is a revised assessment which may be adjusted from a prior assessment due the feedback unless the feedback is "I concur".
"""
        super().__init__(self.system_role, llm)

    def forecast(self, ifps):
        prompt = '^^^'.join([f"FORECAST|{ifp.event}|{ifp.question_id}|{ifp.news}|{ifp.resolution_criteria}|{ifp.background}|{ifp.fine_print}|" for ifp in ifps.values()])
        self.F0 = self.chat(prompt)
        self.F1 = [x.strip().replace('\n', '') for x in self.F0.split('^^^')]
        self.F2 = [x.split('|') for x in self.F1] 
        self.F3 = [[x for x in y if x] for y in self.F2]
        self.F4 = [(int(id),int(forecast),rationale) for _, id, forecast, rationale in self.F3]
        for id, forecast, rationale in self.F4:
            ifps[id].forecast = forecast
            ifps[id].rationale = rationale
            print(id, forecast, rationale)

    def reassess(self, ifp):
        prompt = f"|CRITIC|{ifp.question_id}|{ifp.feedback}|"
        self.R0 = self.chat(prompt)
        id,fcst,rationale = [x for x in self.R0.strip().split('|') if x]
        id = int(id)
        fcst = int(fcst)
        ifps[id].forecast = fcst
        ifps[id].rationale = rationale
        print(id, fcst, rationale)

#### Critic

In [109]:
class Critic (Agent):
    def __init__(self, llm):

        self.system_role = f"""
You a very smart and worldly person reviewing a superforecaster's assignment of probabilities to events.
You will receive an event with probabilities given as |event|id|question|zz|rationale|news|criteria|background|fineprint|.
zz is an integer probability from 1 to 99 and rationale is the student's logic for assigning probability of zz.
You will reply with a line |id|feedback| where feedback is "I concur" if you see no problem with the rationale and zz otherwise presents possible problems with the rationale and zz.
"""
        super().__init__(self.system_role, llm)

    def feedback(self, ifp):
        prompt = f"|{ifp.event}|{ifp.question_id}|{ifp.title}|{ifp.forecast}|{ifp.rationale}|{ifp.news}|{ifp.resolution_criteria}|{ifp.background}|{ifp.fine_print}|"
        self.fb = self.chat(prompt)
        self.fb1 = self.fb.split('|')
        self.fb2 = [x for x in self.fb1 if x]
        try:
            id,feedback = self.fb2
            ifps[int(id)].feedback = feedback
            print(id, feedback)
        except:
            print('problem', self.fb2)
            ifps[int(id)].feedback = 'I concur'

## Summarizer

Add an agent to summarize the back and forth between critic and forecaster into a single cogent rationale that incorporates all that was discussed.

### Forecasting process

In [132]:
def forecasting(ifps, use_newser = False):
    max_tries = 4

    if use_newser:
        ask = Newser()
        ask.research(ifps)
    else:
        analyst = Researcher(ChatGPT)
        analyst.research(ifps)
    
    qr = QuestionRelator(ChatGPT)
    qr.relate(ifps)
    
    sf = Superforecaster(ChatGPT)
    sf.forecast(ifps)
    
    for ifp in ifps.values():
        print("Refining", ifp.question_id)
        ifp.feedback = ''
        critic = Critic(Perplexity)
        for i in range(max_tries):
            print("Pass", i, "of", max_tries, "on", ifp.question_id)
            if 'concur' in ifp.feedback:
                print("concur 1")
                break
            critic.feedback(ifp)
            if 'concur' in ifp.feedback:
                print("concur 2")
                break
            sf.reassess(ifp)
        print("===============================================")

    for ifp in ifps.values():
        print(ifp.question_id, ifp.title)
        print("Forecast", ifp.forecast)
        print("Rationale", ifp.rationale, '\n')

In [89]:
def upload(ifp):
    post_question_prediction(ifp.question_id, ifp.forecast)
    post_question_comment(ifp.question_id, ifp.rationale)

In [90]:
def uploads(ifps):
    for ifp in ifps.values():
        upload(ifp)

## Daily forecast

### Get IFP ids

In [97]:
ifps = list_questions()['results']
today_ids = list(sorted([x['id'] for x in ifps]))
# today_ids = [25876, 25877, 25875, 25873, 25871, 25878, 25874, 25872] # 08JUL24
# today_ids = [26006, 25936, 25935, 25934, 25933, 26004, 26005] # 09JUL24
# today_ids = [25955, 25956, 25957, 25960, 25959, 25954, 25953, 25952, 25958] # 10JUL24
# today_ids = [26019, 26018, 26017, 26020, 26022, 26021, 26023, 26024] # 11JUL24
# today_ids = [26095, 26096, 26097, 26098, 26099, 26100, 26101, 26102] # 12JUL24
# today_ids = [26133, 26134, 26138, 26139, 26140, 26157, 26158, 26159] # 15JUL24
# today_ids = [26189, 26190, 26191, 26192, 26193, 26194, 26195, 26196] # 16JUL24
# today_ids = [26210, 26211, 26212, 26213, 26214, 26215, 26216] # 17JUL24
# today_ids = [26232, 26233, 26234, 26235, 26236] # 18JUL24
# today_ids = [26302, 26303, 26304, 26305, 26306, 26307] # 19JUL24
# today_ids = [26387, 26388, 26389, 26390, 26391, 26392] # 22JUL24
# today_ids = [26404, 26405, 26406, 26407, 26408] # 23JUL24
# today_ids = [26550, 26551, 26552, 26553, 26554, 26555] # 24JUL24
# today_ids = [26568, 26569, 26570, 26571, 26572, 26573, 26574, 26575, 26576, 26577] # 25JUL24
# today_ids = [26638, 26639, 26640, 26641, 26642, 26643, 26644, 26645, 26646] # 26JUL24
# today_ids = [26665, 26666, 26667, 26668, 26669, 26670, 26671, 26683] # 29JUL24
# today_ids = [26700, 26701, 26702, 26703, 26704, 26705, 26706] # 30JUL24
# today_ids = [26816, 26817, 26818, 26819, 26820, 26821, 26844] # 31JUL24
# today_ids = [26771, 26772, 26773, 26774, 26775, 26776, 26777, 26778, 26779, 26780, 26781] # 01AUG24
# today_ids = [26837, 26838, 26839, 26840, 26841, 26842] # 02AUG24

In [98]:
today_ids 

[26837, 26838, 26839, 26840, 26841, 26842]

## Forecast

In [99]:
ifps = {id: IFP(id) for id in today_ids}

In [133]:
forecasting(ifps)

 17%|███████▌                                     | 1/6 [00:02<00:11,  2.28s/it]

26837 Will "Woman's World" by Katy Perry achieve a ranking higher than 15th on the Billboard Hot 100 before October 1, 2024?
I will gather the latest information about the current ranking trajectory of "Woman's World" by Katy Perry on the Billboard Hot 100 since the time of the prompt (July 26, 2024).

Please hold on while I retrieve the latest updates.

---

As of my 2023 knowledge cutoff and hypothetical tools, the latest Billboard Hot 100 ranking can be accessed by checking the [Billboard Hot 100](https://www.billboard.com/charts/hot-100/) chart.

For real-time updates on the ranking of "Woman's World" by Katy Perry, you would need to regularly check the Billboard Hot 100 link provided or follow music industry news sources for any breaking updates on the song’s performance. As per your background information, the song was at 63rd position on July 26, 2024. 

For the final resolution before October 1, 2024, the song must achieve a ranking higher than 15th, and this can be confirmed b

 33%|███████████████                              | 2/6 [00:05<00:12,  3.09s/it]

26838 Will the median net worth on Bloomberg's Billionaires Index be above $10.2 billion on September 16, 2024?
To determine whether the median net worth on Bloomberg's Billionaires Index will be above $10.2 billion on September 16, 2024, I need to track recent movements and trends in the net worths of the world's billionaires as presented on the index.

Please hold on while I gather the latest updates.

---

As of the recent data in July 2024, the median net worth on Bloomberg's Billionaires Index stood at $10.0 billion. This implies an upward trend from $9.535 billion in April 2024 to $10.0 billion in July 2024.

To resolve this question definitively, you will need to check the median net worth on Bloomberg's Billionaires Index at the specific time and date mentioned on September 16, 2024, after 5:00 PM Eastern Time by following this [link](https://www.bloomberg.com/billionaires/).

For continuous updates and to monitor wealthy individuals' positions on the list, regularly visiting t

 50%|██████████████████████▌                      | 3/6 [00:08<00:09,  3.02s/it]

...

 83%|█████████████████████████████████████▌       | 5/6 [00:17<00:03,  3.71s/it]

26841 Will Spirit Airlines file for bankruptcy before October 1, 2024?
To determine if Spirit Airlines is likely to file for bankruptcy before October 1, 2024, I will search for recent news, financial reports, and relevant statements about the company's financial health and status.

Please hold on while I collect the relevant information.

---

Based on the information:

1. **Historical Context**: Spirit Airlines has faced financial difficulties since the start of the COVID-19 pandemic and had a buyout offer from JetBlue rejected by antitrust regulators.
2. **Current Financial Status**: The company continues to struggle with declining revenues and negative earnings and cash flows as of Q2 2024.
3. **Recent News**: Reviewing recent news articles, financial reports, and statements from Spirit Airlines or industry analysts can provide insight into their immediate financial health and outlook.

Specifically:
- **Latest Financial Reports**: Check Spirit Airlines' Q2 2024 and any interim fin

100%|█████████████████████████████████████████████| 6/6 [00:20<00:00,  3.34s/it]

26842 Before October 1, 2024, will OpenAI announce on the news section of its website that it is planning an IPO?
To determine if OpenAI will announce on the news section of its website before October 1, 2024, that it is planning an IPO, I will search for recent news and official statements from OpenAI.

Please hold on while I gather the relevant information.

---

Based on the criteria provided:

1. **Official Announcements**: The announcement must appear in the [news section of OpenAI's website](https://openai.com/news/).
2. **Recent Valuation**: OpenAI was recently valued at $80 billion, and there has been speculation about a potential IPO.
3. **Monitoring**: Continual checking of the news section on the OpenAI website and trusted financial news sources for any updates or rumors about an IPO.

As of now, there is no specific announcement in the news section of OpenAI's website regarding an IPO.

To stay up-to-date:
- Check the [News section of OpenAI's website](https://openai.com/ne

26837 Billboard Rank of "Woman's World"
26838 Bloomberg's Billionaires Index Median Net Worth
26839 Apple iPhone Battery Technology Announcement
26840 Tesla Production Increase
26841 Spirit Airlines Bankruptcy
26842 OpenAI IPO Announcement
26837 15 Given the song's poor initial reception and its current position of 63rd on the Billboard Hot 100 as of July 26, 2024, it seems unlikely the song will climb to a position higher than 15th by October 1, 2024. However, there is a possibility that increased media attention or strategic marketing could boost its performance.
26838 70 The median net worth on Bloomberg's Billionaires Index has shown a rising trend, moving from $9.535 billion in April 2024 to $10.0 billion in July 2024. If this upward trajectory continues, it is plausible that the median net worth could exceed $10.2 billion by September 16, 2024.
26839 40 Although there are rumors suggesting the iPhone 16 Pro models will have stacked battery technology, no official announcement has

## Upload

In [135]:
uploads(ifps)

Prediction posted for  26837
Comment posted for  26837
Prediction posted for  26838
Comment posted for  26838
Prediction posted for  26839
Comment posted for  26839
Prediction posted for  26840
Comment posted for  26840
Prediction posted for  26841
Comment posted for  26841
Prediction posted for  26842
Comment posted for  26842


In [136]:
len(today_ids)

6